# Contenido
* [Introducción](#intro)
* [Descripción del proyecto](#desc)
* [Preprocesamiento de datos](#prep)
* [Análisis exploratorio](#exp)

## Introducción<a id='intro'></a>

Nuestro dataset a consta información acerca de autos usados y nuevos en venta de una compañia en un portal de internet. El objetivo del trabajo es realizar un pequeño preprocesamiento de la información y la realización de un análisis exploratorio, así como la creación de una API WEB desplegado en un servicio en la nube accesible para cualquier persona

## Descripción del proyecto<a id= 'desc'></a>

In [184]:
# Libraries
import pandas as pd
import numpy as np
import plotly.express as px
import streamlit as st

In [185]:
# reading cvs file
df = pd.read_csv('C:/Users/jfcom/Documents/modulo_adicional/proyecto/vehicles_EDA/vehicles_us.csv')

# Show a data resume
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


Contamos con un dataset de 51525 datos con 13 columnas en algunas de ellas tenemos valores nulos. Los datos presnetados son:

* price: Precio en USD
* model_year: Año del modelo
* model: Modelo
* condition: Condición (new, like new, excellent, good, salvage, fair)
* cylinders: Número de cilíndros
* fuel: Tipo de combustible utilizado(gasolina, diesel, etc.)
* odometer: El millaje del vehículo cuando el anuncio fue publicado
* transmission: Tipo de transmisión
* paint_color: Color del auto
* is_4wd: Si el vehículo tiene tracción a las 4 ruedas 
* date_posted: Fecha en la que el anuncio fue publicado
* days_listed: días de la publicación desde el inicio hasta la eliminación.

A continuación mostraremos una parte de los datos, si contamos con datos duplicados y la cantidad de valores nulos por columna.

In [186]:
# Sample of data
df.head(10)

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79
3,1500,2003.0,ford f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28
5,14990,2014.0,chrysler 300,excellent,6.0,gas,57954.0,automatic,sedan,black,1.0,2018-06-20,15
6,12990,2015.0,toyota camry,excellent,4.0,gas,79212.0,automatic,sedan,white,NaN,2018-12-27,73
7,15990,2013.0,honda pilot,excellent,6.0,gas,109473.0,automatic,SUV,black,1.0,2019-01-07,68
8,11500,2012.0,kia sorento,excellent,4.0,gas,104174.0,automatic,SUV,NaN,1.0,2018-07-16,19
9,9200,2008.0,honda pilot,excellent,NaN,gas,147191.0,automatic,SUV,blue,1.0,2019-02-15,17


In [187]:
# Verifying duplicated data
print(f'Total de datos duplicados: {df.duplicated().sum()}')

Total de datos duplicados: 0


In [188]:
# sum of nan values
df.isna().sum()

price               0
model_year       3619
model               0
condition           0
cylinders        5260
fuel                0
odometer         7892
transmission        0
type                0
paint_color      9267
is_4wd          25953
date_posted         0
days_listed         0
dtype: int64

Contamos con un total de de 5 columnas con valores nulos,siendo la que más tiene la columnas `is_4wd`. No existen datos duplicados.
Para este ejercicio eliminaremos la columna `paint_color` ya que no realizaremos algún análsis con esa información

In [189]:
# Erasing paint_color column
df.drop('paint_color', axis=1, inplace=True)

# Showing columns
df.columns

Index(['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel',
       'odometer', 'transmission', 'type', 'is_4wd', 'date_posted',
       'days_listed'],
      dtype='object')

## Preprocesamiento de datos<a id = 'prep'></a>

En está sección trabajaremos con las columnas que tienen valores nulos y cambiaremos los tipos de datos correctos, para ello las estrategias a seguir por columna son:

* **model_year**: Estimaremos los valores agrupando las columnas `model`, `transmission` y `type`, nos encontraremos con casos en que la caracteristicas de modelo, transmisión y tipo de vehículo pueden tener varios años asignados por lo que utilizaremos la moda.
* **cylinders**:  Para esta columna agruparemos por `model_year`, `model`, `transmission` y `type` y realizaremos lo mismo que con la columna `model_year`.
* **odometer**: aquí podemos relacionar para agrupar las columnas `model_year` y `condition`,  llenaremos nuestros valores con el promedio.
* **is_4wd**: Esta columna los valores nulos lo podemos imputar con el valor de False cambiando el tipo de dato a booleano

In [190]:
# Filling Nan Values from model_year colum
df['model_year'] = df.groupby(['model','transmission', 'type'])['model_year']\
    .transform(lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else x.iloc[0]))
    
# Filling Nan Values from cylinders colum
df['cylinders'] = df.groupby(['model_year','model','transmission', 'type'])['cylinders']\
    .transform(lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else x.iloc[0]))
    
# Filling Nan Values from cylinders colum
df['odometer'] = df.groupby(['model_year','condition'])['odometer']\
    .transform(lambda x: x.fillna(x.mean()))\
    .round()

# Filling Nan Values from is_4wd colum
df['is_4wd']=df['is_4wd'].astype('bool')
#df['is_4wd'].fillna(0, inplace=True)

# verifying Nan values
df.isna().sum()

price             0
model_year       19
model             0
condition         0
cylinders       218
fuel              0
odometer         26
transmission      0
type              0
is_4wd            0
date_posted       0
days_listed       0
dtype: int64

Aún nos quedan algunos valores nulos que podemos eliminar, primero comprobemos que la eliminación de estos no nos eliminé mucha información del dataset original.

In [191]:
# Proving data reduction
print(f'porcentaje de reducción de dataset: {len(df.dropna())/len(df) :0.2%}')


porcentaje de reducción de dataset: 99.56%


perdemos menos de del 1% de nuestros datos por lo que eliminaremos los valores nulos faltantes y realizaremos los cambios de los tipos correctos.

In [192]:
# Erasing Nan values
df.dropna(inplace= True)

# Changing values to int type
col = ['model_year', 'cylinders', 'odometer']

for element in col:
    df[element]=df[element].astype('Int64')
    
# Changing values to DateTime type
df['date_posted']=df['date_posted'].astype('datetime64[ns]')


Por último verificaremos que se hayan realizando todos los cambios realizados.

In [193]:
# Show a data resume
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51300 entries, 0 to 51524
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   price         51300 non-null  int64         
 1   model_year    51300 non-null  Int64         
 2   model         51300 non-null  object        
 3   condition     51300 non-null  object        
 4   cylinders     51300 non-null  Int64         
 5   fuel          51300 non-null  object        
 6   odometer      51300 non-null  Int64         
 7   transmission  51300 non-null  object        
 8   type          51300 non-null  object        
 9   is_4wd        51300 non-null  bool          
 10  date_posted   51300 non-null  datetime64[ns]
 11  days_listed   51300 non-null  int64         
dtypes: Int64(3), bool(1), datetime64[ns](1), int64(2), object(5)
memory usage: 4.9+ MB


In [194]:
# Sample of data
df.head(10)

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,is_4wd,date_posted,days_listed
0,9400,2011,bmw x5,good,6,gas,145000,automatic,SUV,True,2018-06-23,19
1,25500,2010,ford f-150,good,6,gas,88705,automatic,pickup,True,2018-10-19,50
2,5500,2013,hyundai sonata,like new,4,gas,110000,automatic,sedan,True,2019-02-07,79
3,1500,2003,ford f-150,fair,8,gas,201481,automatic,pickup,True,2019-03-22,9
4,14900,2017,chrysler 200,excellent,4,gas,80903,automatic,sedan,True,2019-04-02,28
5,14990,2014,chrysler 300,excellent,6,gas,57954,automatic,sedan,True,2018-06-20,15
6,12990,2015,toyota camry,excellent,4,gas,79212,automatic,sedan,True,2018-12-27,73
7,15990,2013,honda pilot,excellent,6,gas,109473,automatic,SUV,True,2019-01-07,68
8,11500,2012,kia sorento,excellent,4,gas,104174,automatic,SUV,True,2018-07-16,19
9,9200,2008,honda pilot,excellent,6,gas,147191,automatic,SUV,True,2019-02-15,17


## Análisis exploratorio<a id = 'exp'></a>